<a href="https://colab.research.google.com/github/JonasCGN/project_neural_network/blob/main/project_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Visão Computacional: O Guia Completo - Redes Neurais para cliassificação de imagens


##Abordagem 1: extração de todos os pixels da imagem

###Importação das bibliotecas





In [ ]:
import cv2
import numpy as np
import os
import zipfile
from google.colab.patches import cv2_imshow
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!pip install tensorflow == 2.6.0

###Extração dos pixels das imagens





In [ ]:
path = '/content/drive/MyDrive/Visao_Computacional_Guia_Completo/Datasets/homer_bart_1.zip'
zip_object = zipfile.ZipFile(file=path, mode = 'r')
zip_object.extractall('./')
zip_object.close()

In [ ]:
diretorio = '/content/homer_bart_1'
arquivos = [os.path.join(diretorio,f) for f in sorted(os.listdir(diretorio))]
print(arquivos)

['/content/homer_bart_1/.DS_Store', '/content/homer_bart_1/bart1.bmp', '/content/homer_bart_1/bart10.bmp', '/content/homer_bart_1/bart100.bmp', '/content/homer_bart_1/bart101.bmp', '/content/homer_bart_1/bart102.bmp', '/content/homer_bart_1/bart103.bmp', '/content/homer_bart_1/bart104.bmp', '/content/homer_bart_1/bart105.bmp', '/content/homer_bart_1/bart106.bmp', '/content/homer_bart_1/bart108.bmp', '/content/homer_bart_1/bart109.bmp', '/content/homer_bart_1/bart11.bmp', '/content/homer_bart_1/bart110.bmp', '/content/homer_bart_1/bart111.bmp', '/content/homer_bart_1/bart112.bmp', '/content/homer_bart_1/bart113.bmp', '/content/homer_bart_1/bart114.bmp', '/content/homer_bart_1/bart115.bmp', '/content/homer_bart_1/bart116.bmp', '/content/homer_bart_1/bart117.bmp', '/content/homer_bart_1/bart118.bmp', '/content/homer_bart_1/bart119.bmp', '/content/homer_bart_1/bart12.bmp', '/content/homer_bart_1/bart120.bmp', '/content/homer_bart_1/bart121.bmp', '/content/homer_bart_1/bart122.bmp', '/conte

In [ ]:
largura, altura = 128,128

In [ ]:
# arquivos.remove('/content/homer_bart_1/.DS_Store')
imagens = []
classes = []

for imagem_caminho in arquivos:
  try:
    imagem = cv2.imread(imagem_caminho)
    (H,W) = imagem.shape[:2]
  except:
    continue

  imagem = cv2.resize(imagem, (largura,altura))
  imagem = cv2.cvtColor(imagem, cv2.COLOR_BGR2GRAY)

  imagem = imagem.ravel()
  imagens.append(imagem)

  nome_imagem = os.path.basename(os.path.normpath(imagem_caminho))
  bart = nome_imagem.startswith('b')
  classe = 1
  if (bart):
    classe = 0

  classes.append(classe)


In [ ]:
x = np.asarray(imagens)
y = np.asarray(classes)

In [ ]:
np.unique(y, return_counts=True)

(array([0, 1]), array([160, 109]))

###Normalização dos dados





In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x = scaler.fit_transform(x)

###Bases de treinamento e teste





###Construção e treinamento da rede neural





###Avaliação da rede neural



